## Train regression models for demand forecasting

This script trains regression models using 
- **Weather data** (historical)
- **Demand data** (historical)

to predict demand based on:
- **Weather data** (historical / future)

### Import Packages

In [7]:
import numpy as np
import pyarrow.parquet as pq
import joblib
import scipy
import pandas as pd
import time
import os
import re
import glob
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import sklearn
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor as MLP
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta
import seaborn as sns
import pytz
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
import yaml
import pprint
import pwlf

from src import input_ops
from src import model_ops

### Load config file with scenarios and parameters 

In [ ]:
config_file_name = 'config1'; config_path = f"config/{config_file_name}.yaml"; config = input_ops.load_config(config_path)

aggregation_level = config['aggregation_level']
smart_ds_year = config['smart_ds_years'][0]
building_types = config["building_types"]
input_data_dict_name = config['input_data_dict_name']
Y_column = config['Y_column']
X_columns = config['X_columns']

## Initialize parameters for saving paths
output_path_str = config['output_data_training_path']
aggregation_level = config['aggregation_level']

if Y_column == 'cooling_kw_sum':
    prediction_model_str = config['prediction_model_cooling']
elif Y_column == 'heating_kw':
    prediction_model_str = config['prediction_model_heating']
else:
    raise ValueError('Model architecture is not defined for model output! (only for cooling_sum and heating)')
    
print(f"Aggregation level: {config['aggregation_level']} \n SMART-DS year: {config['smart_ds_years'][0]} \n Months: {config['start_month']}-{config['end_month']} \n output: {config['Y_column']}, \n inputs: {config['X_columns_set']} \n Prediction model: {prediction_model_str} \n output directory:{config['output_data_training_path']}")

### Load input data (Resstock weather data & smart-ds load data)

In [ ]:
## Load input load & weather data of year-city-region-building_type combinations
loaded_input_data_dict = {}
if config['aggregation_level'] == 'building': # Assuming buildings have separate joblib files
    for city, region in CITY_REGIONS_TO_RUN.items():
        path_temp = os.path.join(config["input_data_training_path"], f"{city}/{region}/{input_data_dict_name}.joblib")
        print(path_temp)
else:
    print(f'Loading dictionary with load & weather data from {config["input_data_training_path"]}')
    loaded_input_data_dict = joblib.load(os.path.join(config["input_data_training_path"], f"{input_data_dict_name}.joblib")) # Load dictionary with load & weather data 

### Train ML models and save models, scalers and metrics
---

In [1]:
## Train MLP model for year-city-region-building_type combinations
start_time = time.time()

## Initialize dictionaries
mlp_results = {} # change to mlp_metrics_dict
mlp_model_dict = {}
xnorm_dict = {}
ynorm_dict = {}

for smart_ds_year, city, region, load_model, building_type in loaded_input_data_dict.keys():
    print(f".......Training {prediction_model_str} for {smart_ds_year} {city} {region} {load_model} {building_type}.......")
    # Train MLP model
    # Load data
    if not config['aggregation_level'] == 'building':
        input_df = loaded_input_data_dict[(smart_ds_year, city, region, load_model, building_type)]
    else:
        print("To train model at the building level: (1) create processed input data at the building level, and (2) update code here to load it")

    match prediction_model_str:
        case 'MLP_2_1452_10':
            first_L_n = 1452
            second_L_n = 10
            MLP_model, xnorm, ynorm, y_test, y_test_pred, y_train, y_train_pred, training_runtime = model_ops.train_mlp_model(input_df, X_columns, Y_column, first_L_n, second_L_n)
        case 'MLP_2_4096_512': 
            first_L_n = 2048
            second_L_n = 256
            MLP_model, xnorm, ynorm, y_test, y_test_pred, y_train, y_train_pred, training_runtime = model_ops.train_mlp_model(input_df, X_columns, Y_column, first_L_n, second_L_n) 
        case 'MLP_2_2048_512': 
            first_L_n = 4096
            second_L_n = 512
            MLP_model, xnorm, ynorm, y_test, y_test_pred, y_train, y_train_pred, training_runtime = model_ops.train_mlp_model(input_df, X_columns, Y_column, first_L_n, second_L_n)    
        case 'MLP_2_1024_128': 
            first_L_n = 1024
            second_L_n = 128
            MLP_model, xnorm, ynorm, y_test, y_test_pred, y_train, y_train_pred, training_runtime = model_ops.train_mlp_model(input_df, X_columns, Y_column, first_L_n, second_L_n)
        case 'MLP_2_512_64': 
            first_L_n = 512
            second_L_n = 64
            MLP_model, xnorm, ynorm, y_test, y_test_pred, y_train, y_train_pred, training_runtime = model_ops.train_mlp_model(input_df, X_columns, Y_column, first_L_n, second_L_n)
        case 'MLP_2_256_32':
            first_L_n = 256
            second_L_n = 32
            MLP_model, xnorm, ynorm, y_test, y_test_pred, y_train, y_train_pred, training_runtime = model_ops.train_mlp_model(input_df, X_columns, Y_column, first_L_n, second_L_n)
        case 'MLP_2_100_10':
            first_L_n = 100
            second_L_n = 10
            MLP_model, xnorm, ynorm, y_test, y_test_pred, y_train, y_train_pred, training_runtime = model_ops.train_mlp_model(input_df, X_columns, Y_column, first_L_n, second_L_n)
        case 'MLP_2_32_16':
            first_L_n = 32
            second_L_n = 16
            MLP_model, xnorm, ynorm, y_test, y_test_pred, y_train, y_train_pred, training_runtime = model_ops.train_mlp_model(input_df, X_columns, Y_column, first_L_n, second_L_n)
        case 'MLP_1_16':
            first_L_n = 16
            second_L_n = ''
            MLP_model, xnorm, ynorm, y_test, y_test_pred, y_train, y_train_pred, training_runtime = model_ops.train_mlp_model(input_df, X_columns, Y_column, first_L_n, second_L_n)
        case 'PWLR_4bp':
            # X_columns = ['Dry Bulb Temperature [°C]']
            MLP_model, xnorm, ynorm, y_test, y_test_pred, training_runtime = model_ops.train_piecewise_linear_model(input_df, X_columns, Y_column, num_breakpoints=4)
        case 'Poly_4deg':
            MLP_model, poly_transform, xnorm, ynorm, y_test, y_test_pred, training_runtime = model_ops.train_polynomial_model(input_df, X_columns, Y_column, degree=4, alpha=1.0)
        case _:
            MLP_model, xnorm, ynorm, y_test, y_test_pred, training_runtime = model_ops.train_linear_model(input_df, X_columns, Y_column)

    # compute MLP performance metrics
    r2, mae, mape, rmse, nrmse, peak_load_error, peak_time_error, mape_high, nrmse_high = model_ops.metrics_test_data(y_test, y_test_pred)

    # Save MLP performance metrics
    mlp_results[(smart_ds_year, city, region, load_model, building_type)] = {
        "X_columns": X_columns,
        "Y_column": Y_column,
        "training_runtime": training_runtime,
        "y_test": y_test, 
        "y_test_pred": y_test_pred,
        "y_train": y_train, 
        "y_train_pred": y_train_pred,
        "r2": r2,
        "mae": mae,
        "mape": mape,
        "rmse": rmse,
        "nrmse": nrmse,
        "peak_load_error": peak_load_error,
        "peak_time_error": peak_time_error,
        "mape_high": mape_high,
        "nrmse_high": nrmse_high
    }
    
    # Save trained models and scalers in dictionaries
    mlp_model_dict[(smart_ds_year, city, region, load_model, building_type)] = MLP_model
    xnorm_dict[(smart_ds_year, city, region, load_model, building_type)] = xnorm
    ynorm_dict[(smart_ds_year, city, region, load_model, building_type)] = ynorm

# Save dictionaries as joblib files
joblib.dump(mlp_results, os.path.join(output_path_str, "metrics", f"{prediction_model_str}_results.joblib"))
joblib.dump(mlp_model_dict, os.path.join(output_path_str, "models", f"{prediction_model_str}_models_dict.joblib"))
joblib.dump(xnorm_dict, os.path.join(output_path_str, "scalers", f"{prediction_model_str}_xnorm_dict.joblib"))
joblib.dump(ynorm_dict, os.path.join(output_path_str, "scalers", f"{prediction_model_str}_ynorm_dict.joblib"))

print("All models, scalers, and results saved successfully!")

end_time = time.time(); print(f"Runtime for Training models: {(end_time - start_time) / 60:.2f} minutes")